In [1]:
from codebase.utils import MongoConnector

In [2]:
conn = MongoConnector("./config.ini")

In [3]:
def make_text_query(keyword):
    
    q = { 
            "$text" : { 
                "$search" : f"\"{keyword}\"", 
                "$language" : "en"
            }
    }
    
    return q

In [4]:
# keywords = ["Old","Age","Facility","Nursing home","Old people","Elderly","Boomer",\
#             "Ageism","Older","Ageist","Senior","Senior citizen","Retired people",\
#             "Older adults","Elder","Old lady","Old man","Grandma","Grandpa","Granny",\
#             "Over the hill","Cougar","Grumpy","Old coot","Geezer","Fogey","Codger"]

keywords = ["Old people", "Elderly", "Ageism", "Ageist", "Older adults", "Grumpy old people"]
Weeks = ["FirstWeek_March", "SecondWeek_March", "ThirdWeek_March", "FourthWeek_March",\
        "FirstWeek_April", "SecondWeek_April", "ThirdWeek_April", "FourthWeek_April"]

In [ ]:
num_count = dict()
for keyword in keywords:
    keyword = keyword.lower()
    num_count[keyword] = 0
    flag = keyword.replace(" ", "-")
    wf = open(f"export/{flag}-tweets.tsv", "w")
    wf.write("No\tCreated_at\tText\n")
    for each_week in Weeks:
        conn.get_collection_cursor(each_week)
        docs = conn.target_collection.find(make_text_query(keyword),\
                                          no_cursor_timeout=True)
        for i, doc in enumerate(docs):
            if i % 1000 == 0:
                print(f"{keyword}@{each_week}: {i}")
            num_count[keyword] += 1
            created_at = doc["created_at"]
            text = doc["text"].replace("\n", " ")
            wf.write(f"{i}\t{created_at}\t{text}\n")
    
    wf.close()
    cnt = num_count[keyword]
    print(f"Finished {keyword}: {cnt} tweets.")